**<font color = black size=6>实验四:决策树(2)</font>**

**<font color = blue size=4>第一部分:实验任务</font>**

本次实验承接上次实验，用【ID3】算法实现一棵完整的决策树。

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
from collections import Counter

<span style="color:purple">1) 整理上次实验的代码，编写函数，【从属性集A中】寻找使得信息增益最大的属性   
    输入：数据集D、属性集A   
    输出：最佳划分的属性(维度)     
    计算信息增益公式:  
    某数据集D有若干特征值以及对应的标签值，其总样本大小为|D|,这里取其中一个特征类型feature,该特征包含V个不同的取值，特征值为第v(v=1,2,...,V)个值的数量为|$D^v$|$(\sum_{v=1}^VD^v=|D|)$,则该特征值对应的信息增益为$$Gain(D,feature)=Ent(D)-\sum_{v=1}^K \frac{|D^v|}{D} Ent(D^v)$$</span>

In [6]:
def entropy(label):
    label = label.reshape(len(label),1)
    counter = Counter(label[:,0])
    a=np.unique(label[:,0])
    ent=0
    m = len(label)
    for i in range(len(a)):
        ent -= counter[a[i]]/m*np.log2(counter[a[i]]/m)
    return ent

def split(feature, label, dimension):
    label = label.reshape(len(label),1)
    a=np.unique(feature[:,dimension])
#     split_feature = np.empty(len(a))
    split_feature = []
    split_label = []
    for i in range(len(a)):
        tem1 = []
        tem2 = []
        for j in range(len(label)):
            if feature[j,dimension]==a[i]:
                tem1.append(feature[j,:])
                tem2.append(label[j,:])
        tem1 = np.array(tem1)
        tem2 = np.array(tem2)
        split_feature.append(tem1)
        split_label.append(tem2)
    split_feature = np.array(split_feature)
    split_label = np.array(split_label)
    return split_feature,split_label


def best_split(D, A):
    best_entropy = -100
    best_dimension = -1
    ladi = D.shape[1]-1
    
    tot = entropy(D[:,ladi])
    for i in A:
        tem = tot
        split_feature,split_label=split(D,D[:,D.shape[1]-1],i)
        for j in range(len(split_label)):
            tem-=entropy(split_label[j])
        if tem>best_entropy:
            best_entropy=tem
            best_dimension=i
    
        
    return best_dimension

def same(D,A):
    tem = np.sum(D,axis=0)/len(D)
    for i in A:
        if all(D[:,i] != tem[i]):
            return False
    return True

<span style="color:purple">2) 完成DTree类中的TreeGenerate、train函数以完成决策树的构建。并完成DTree类中的predict函数来用构建好的决策树来对测试数据集进行预测并输出预测准确率。</span>

In [7]:
# 树结点类
class Node:
    def __init__(self, isLeaf=True, label=-1, index=-1):
        self.isLeaf = isLeaf # isLeaf表示该结点是否是叶结点
        self.label = label # label表示该叶结点的label（当结点为叶结点时有用）
        self.index = index # index表示该分支结点的划分属性的序号（当结点为分支结点时有用）
        self.children = {} # children表示该结点的所有孩子结点，dict类型，方便进行决策树的搜索
        
    def addNode(self, val, node):
        self.children[val] = node #为当前结点增加一个划分属性的值为val的孩子结点

In [8]:
# 决策树类
class DTree:
    def __init__(self):
        self.tree_root = None #决策树的根结点
        self.possible_value = {} # 用于存储每个属性可能的取值
    
        
    '''
    TreeGenerate函数用于递归构建决策树，伪代码参照课件中的“Algorithm 1 决策树学习基本算法”
    '''
    def TreeGenerate(self, D, A):

        # 生成结点 node
        node = Node()
        
        ladi = D.shape[1]-1
        counter = Counter(D[:,ladi])
        # if D中样本全属于同一类别C then
        #     将node标记为C类叶结点并返回
        # end if
        if counter[0] == len(D):
            node.label = 0
            node.isLeaf = True
            return node
        elif counter[1] == len(D):
            node.label = 1
            node.isLeaf = True 
            return node
        
        
        
        # if A = Ø OR D中样本在A上取值相同 then
        #     将node标记叶结点，其类别标记为D中样本数最多的类并返回
        # end if
        
        if (len(A)==0) or same(D,A):
            
            if counter[0]>counter[1]:
                node.label = 0
            else:
                node.label = 1
            node.isLeaf = True
            return node
        
        
        
        
        # 从A中选择最优划分属性a_star
        # （选择信息增益最大的属性，用到上面实现的best_split函数）
      
        a_star = best_split(D, A)
        
        
        # for a_star 的每一个值a_star_v do
        #     为node 生成每一个分支；令D_v表示D中在a_star上取值为a_star_v的样本子集
        #     if D_v 为空 then
        #         将分支结点标记为叶结点，其类别标记为D中样本最多的类
        #     else
        #         以TreeGenerate(D_v,A-{a_star}) 为分支结点
        #     end if
        # end for
        tem = Counter(D[:,a_star])
        tem1 = np.unique(D[:,a_star])
        node.isLeaf = False
        node.index = a_star
        for a_star_v in tem1:
            newnode = Node(False)
            if(tem[a_star_v]==0):
                newnode.isleaf = True
                newnode.label = 0           ############
            else:
                D_v = []
                for i in range(len(D)):
                    if D[i,a_star] == a_star_v:
                        D_v.append(D[i,:])
                D_v = np.array(D_v)
                if(len(A)!=0):     ##############
                    A.remove(a_star)
                newnode = self.TreeGenerate(D_v,A)
            node.children[a_star_v] = newnode
        
        
        return node
    
    
    
    
    '''
    train函数可以做一些数据预处理（比如Dataframe到numpy矩阵的转换，提取属性集等），并调用TreeGenerate函数来递归地生成决策树
    '''
    def train(self, D):
        D = np.array(D) # 将Dataframe对象转换为numpy矩阵（也可以不转，自行决定做法）
        A = set(range(D.shape[1] - 1)) # 属性集A
        A = list(A)
#         #记下每个属性可能的取值
        for every in A:
            self.possible_value[every] = np.unique(D[:, every])
        
        self.tree_root = self.TreeGenerate(D, A) # 递归地生成决策树，并将决策树的根结点赋值给self.tree_root
        
        
        
    
    
    
    
    '''
    predict函数对测试集D进行预测， 并输出预测准确率（预测正确的个数 / 总数据数量）
    '''
    def predict(self, D):
        D = np.array(D) # 将Dataframe对象转换为numpy矩阵（也可以不转，自行决定做法）
        correct = 0
#         #对于D中的每一行数据d，从当前结点x=self.tree_root开始，当当前结点x为分支结点时，
#         #则搜索x的划分属性为该行数据相应的属性值的孩子结点（即x=x.children[d[x.index]]），不断重复，
#         #直至搜索到叶结点，该叶结点的label就是数据d的预测label
        for i in range(len(D)):
            x = self.tree_root
            d = D[i,:]
            while x.isLeaf == False:
                x=x.children[d[x.index]]
            if x.label == d[len(d)-1]:
                correct += 1
        return correct/len(D)


In [9]:
train_frame = pd.read_csv('train_titanic.csv')
dt = DTree()

# 构建决策树
dt.train(train_frame)

# 利用构建好的决策树对测试数据集进行预测，输出预测准确率（预测正确的个数 / 总数据数量）
test_frame = pd.read_csv('test_titanic.csv')
dt.predict(test_frame)

0.7821782178217822

**<font color = blue size=4>第二部分:作业提交</font>**

一、实验课下课前提交完成代码，如果下课前未完成，请将已经完成的部分进行提交，未完成的部分于之后的实验报告中进行补充  
要求:  
1)文件格式为：学号-姓名.ipynb  
2)【不要】提交文件夹、压缩包、数据集等无关文件，只需提交单个ipynb文件即可，如果交错请到讲台前联系助教，删掉之前的错误版本后再进行提交

二、这两周的实验课内容汇总到同一个实验报告中，于下周五实验课(4月1号前)上课前提交报告  
要求：  
1)文件格式为：学号-姓名.pdf  
2)【不要】提交文件夹、压缩包、代码文件、数据集等任何与实验报告无关的文件，只需要提交单个pdf文件即可  
3)文件命名时不需要额外添加“实验几”等额外信息，按照格式提交  
4)每周的实验报告提交地址会变化，且有时间限制，提交时间为下周的实验课开始时，请注意及时提交。

实验四(决策树)的实验报告上交地址:https://workspace.jianguoyun.com/inbox/collect/fdadf5486c654f4caf451e1d2c019c07/submit